In [1]:
# Import the necessary libraries
import pyspark.pandas as ps
#------------------do not touch------------------
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import  Vectors
from pyspark.sql.types import DoubleType
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors

from pyspark.sql.functions import sum as spark_sum, count as spark_count
#------------------------------------------------

# TODO: Figure out why we always need to restart the kernel... probably due to an import...


/opt/homebrew/lib/python3.10/site-packages/pyspark/pandas/__init__.py:49: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
# Spark Initialization + Configuration of UDFs

#Initialize a spark session
spark = SparkSession.builder \
    .appName("recommenderTest") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.memory", "7g") \
    .config("spark.sql.shuffle.partitions", "32") \
    .config("spark.sql.pivotMaxValues", "20000") \
    .config("spark.master", "local[*]") \
    .config("spark.sql.codegen.wholeStage", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

def pearson_average(v):
    sum_nonzero = sum(v)
    count_nonzero = len([e for e in v if e != 0])
    mean = sum_nonzero / count_nonzero
    v2 = [e - mean if e != 0 else 0 for e in v] # now subtract the mean from each non zero element
    return Vectors.dense(v2)

def co_sym (x, y):
    pearson1 = x
    pearson2 = y
    return float(pearson1.dot(pearson2)/(Vectors.norm(pearson1,2)*Vectors.norm(pearson2,2)))

dot_udf = udf(co_sym, DoubleType())
spark.udf.register("dot_udf", dot_udf)

23/03/24 18:09:17 WARN Utils: Your hostname, Martin-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.53 instead (on interface en0)
23/03/24 18:09:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/24 18:09:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<function __main__.co_sym(x, y)>

In [3]:
# File Loading

df = spark.read.csv("data/ratings_small.csv", header=True, inferSchema=True)
# File FORMAT: userId, movieId, rating, timestamp
df = df.drop("timestamp")

# Data Splitting
training, test = df.randomSplit([0.8, 0.2])
training.show(10)
test.show(10)
# ==============DF Initialization (to be used later...)==============

# df_user_movie_rating.show(10, 10)
df = training
df_user_movie_rating = df

# ==============Similarity Matrix==============
# Group by movieId and pivot the userId column
df = df.groupBy("movieId").pivot("userId").agg({"rating": "first"}).fillna(0)
df = df.sort("movieId")
df.show(10)

# Assemble the columns into a vector column
assembler = VectorAssembler(inputCols=df.columns[1:], outputCol="features")
df_vector = assembler.transform(df).select('movieId', 'features')
df_vector = df_vector.repartition(10)

# Compute the similarity matrix using the dot product of normalized vectors
similarity_matrix = df_vector.alias("a").crossJoin(df_vector.alias("b")) \
    .where("a.movieId != b.movieId") \
    .selectExpr("a.movieId as movieId", "b.movieId as movieId_1",
                "dot_udf(a.features, b.features) as similarity")
similarity_matrix.show(10, 10)

## Pivot the similarity matrix
# similarity_matrix = similarity_matrix.groupBy("movieId").pivot("movieId_1").agg({"similarity": "first"}).fillna(0)
# similarity_matrix = similarity_matrix.sort("movieId")
# similarity_matrix.select(similarity_matrix.columns[:10]).show(truncate=False)


+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|     31|   2.5|
|     1|   1029|   3.0|
|     1|   1061|   3.0|
|     1|   1129|   2.0|
|     1|   1287|   2.0|
|     1|   1293|   2.0|
|     1|   1339|   3.5|
|     1|   1343|   2.0|
|     1|   1371|   2.5|
|     1|   1405|   1.0|
+------+-------+------+
only showing top 10 rows

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|   1172|   4.0|
|     1|   1263|   2.0|
|     1|   1953|   4.0|
|     1|   2193|   2.0|
|     1|   2294|   2.0|
|     1|   2968|   1.0|
|     1|   3671|   3.0|
|     2|     47|   4.0|
|     2|    185|   3.0|
|     2|    186|   3.0|
+------+-------+------+
only showing top 10 rows



+-------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---

+-------+---------+----------+
|movieId|movieId_1|similarity|
+-------+---------+----------+
|  27410|     7650|       0.0|
|  27410|     5422|       0.0|
|  27410|      247|       0.0|
|  27410|     6643|       0.0|
|  27410|    98607|       0.0|
|  27410|   110655|       0.0|
|  27410|     3355|0.41917...|
|  27410|     1993|       0.0|
|  27410|   110453|       0.0|
|  27410|    61352|       0.0|
+-------+---------+----------+
only showing top 10 rows



In [4]:
# ==============Prediction V2 (STANDBY - DON'T TOUCH FOR NOW)==============

# Note: this code is working but it's taking around 16min to join the similarity matrix with the user_movie_rating matrix
# and then ~1-2sec to predict the rating for each user-movie pair


#--------------------------- CODE (BELOW) ---------------------------

# predict ratings for user_movie_ratings df
df_user_movie_rating = df_user_movie_rating.join(similarity_matrix, df_user_movie_rating.movieId == similarity_matrix.movieId, how='left').drop(similarity_matrix.movieId)
df_user_movie_rating = df_user_movie_rating.withColumnRenamed("similarity.movieId", "movie2")



#------------------do not edit/move------------------
# TODO: figure out this... probably the reason why we need to restart kernel.. must be messing up with other imports...
from pyspark.sql.functions import col, sum
from pyspark.sql.types import FloatType
from pyspark.sql.functions import array, col, sort_array
#------------------------------------------------
test.show(10)
df_user_movie_rating.show(10)
print('----------')
test_df_alias = test.alias("tst")
sm_alias = df_user_movie_rating.alias("sm")
from pyspark.sql import Window
from pyspark.sql.functions import row_number

window_spec = Window.partitionBy("tst.userId", "tst.movieId").orderBy(col("sm.similarity").desc())

joined_df = test_df_alias.join(sm_alias, (col("tst.userId") == col("sm.userId")) & (col("tst.movieId") == col("sm.movieId_1"))).withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= 2)
weighted_df = joined_df.withColumn("weighted_rating", col("sm.rating") * col("sm.similarity"))

from pyspark.sql.functions import sum, expr
from pyspark.sql.functions import coalesce, lit

result_df = (weighted_df
            .groupBy("tst.userId", "tst.movieId", "tst.rating")
            .agg(sum("weighted_rating").alias("sum_weighted_rating"), sum("sm.similarity").alias("sum_similarity"))
            .withColumn("predicted_rating", expr("sum_weighted_rating / sum_similarity"))
            .drop("sum_weighted_rating", "sum_similarity")).withColumn("predicted_rating", coalesce(col("predicted_rating"), lit(0.0)))

result_df.show(10)

# Evaluation

from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="predicted_rating")
rmse = evaluator.evaluate(result_df)
print("Root Mean Squared Error (RMSE) on test data = {:.4f}".format(rmse))

# def predict_user_rating(user_id, movie_id, similarity_matrix):
#     print("Debug -- userID: ", user_id, " movieID: ", movie_id)
#     # Filter similarity matrix to include only ratings for the given user and similar movies
#     user_ratings = similarity_matrix.filter((col("userId") == user_id) & (col("movieId_1") == movie_id))
#     # Sort the ratings by similarity in descending order and select the top 2 most similar movies
#     user_ratings = user_ratings.sort(col("similarity").desc()).limit(2)
#     # Calculate the predicted rating by computing a weighted average of the user's ratings for similar movies
#     user_ratings = user_ratings.withColumn("weighted_rating", (col("rating") * col("similarity")).cast(FloatType()))
#     numerator = user_ratings.agg(sum("weighted_rating")).collect()[0][0]
#     denominator = user_ratings.agg(sum("similarity")).collect()[0][0]
    
#     if denominator != 0:
#         predicted_rating = numerator / denominator
#     else:
#         predicted_rating = None
#     return predicted_rating


# predict_udf = udf(predict_user_rating, FloatType())
# spark.udf.register("predict_udf", predict_udf)
# result_df = test.withColumn("predicted_rating", predict_udf(col("userId"), col("movieId"), df_user_movie_rating))
# result_df.show()

# df_user_movie_rating.show(10)

# predicted_rating1 = predict_user_rating(5,1)
# predicted_rating1.show()
# test.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|   1172|   4.0|
|     1|   1263|   2.0|
|     1|   1953|   4.0|
|     1|   2193|   2.0|
|     1|   2294|   2.0|
|     1|   2968|   1.0|
|     1|   3671|   3.0|
|     2|     47|   4.0|
|     2|    185|   3.0|
|     2|    186|   3.0|
+------+-------+------+
only showing top 10 rows



+------+-------+------+---------+-------------------+
|userId|movieId|rating|movieId_1|         similarity|
+------+-------+------+---------+-------------------+
|     1|   1343|   2.0|    27410|                0.0|
|     1|   1343|   2.0|     7650|                0.0|
|     1|   1343|   2.0|     5422|                0.0|
|     1|   1343|   2.0|      247|0.34741138894027673|
|     1|   1343|   2.0|     6643| 0.1640839079442483|
|     1|   1343|   2.0|    98607|                0.0|
|     1|   1343|   2.0|   110655|                0.0|
|     1|   1343|   2.0|     3355| 0.1079380486581311|
|     1|   1343|   2.0|     1993|0.12367848282062127|
|     1|   1343|   2.0|   110453|0.17490778777865043|
+------+-------+------+---------+-------------------+
only showing top 10 rows

----------


+------+-------+------+------------------+
|userId|movieId|rating|  predicted_rating|
+------+-------+------+------------------+
|     1|   1263|   2.0|1.5562334686533064|
|     2|    356|   3.0|3.5103328595944667|
|     2|    539|   3.0|3.0000000000000004|
|     3|   3510|   4.0|   3.2593287493575|
|     4|    289|   4.0| 4.443671285123623|
|     4|   1033|   5.0|               5.0|
|     4|   1258|   5.0| 4.999999999999999|
|     4|   1580|   5.0| 4.999999999999999|
|     4|   2403|   4.0| 4.536344334797898|
|     5|   1544|   3.5| 3.749821271048303|
+------+-------+------+------------------+
only showing top 10 rows



Root Mean Squared Error (RMSE) on test data = 1.0137
